**Founding principles** of Lasagna AI are:

1. We want to build **layered** agents!
2. We want it to be **pluggable** (both _models_ and _agents_ plug together in all directions).
3. We want to deploy stuff into **production**!
4. We want **type safety**!

## Prerequisite Knowledge

### Python `asyncio`

Lasagna AI is production-focused and fully async, so it plays nicely with remote APIs and modern Python web frameworks. If `asyncio` is new to you, read [Intro to Python Asyncio](misc/python_asyncio.ipynb).

### Functional Programming

The pipeline nature of AI systems lends itself to **functional programming**. If functional programming is new to you, watch [Dear Functional Bros](https://youtu.be/nuML9SmdbJ4?si=eQ6Qla11k3ayJD79) and read [Functional Programming](misc/functional_programming.ipynb).

A quick recap of functional programming:

- State is immutable:
    - Want to modify something? **TOO BAD!**
    - Instead, make a copy (_with your modifications applied_).
- Pass lots of functions as parameters to other functions:
    - We think it's **fun** and **cool**.
    - You will too once you get used to the idea.

### Python Type Hints

(aka, type _annotations_)

Lasagna AI is **100% type hinted**, so take advantage of that!

That is, you should be using a tool like [mypy](https://mypy-lang.org/) or [pyright](https://microsoft.github.io/pyright/) in your project. Why? **Because it will yell at you when you use Lasagna wrong!** That is very useful.

Setting up static type checking may seem tedious, but Lasagna's complex data types make type checking essential — it will save you significant debugging time.

If Python type hints are new to you, read [Intro to Python Type Hints](misc/python_type_hints.ipynb).

### The Python `TypedDict`

Speaking of type hints and productionalization, Lasagna AI uses _lots_ of `TypedDict`s.

A `TypedDict`, at runtime, is just a Python `dict`.

However, during static type checking, it must satisfy a fixed schema (certain keys with certain types of values).

Why all the `TypedDict`s? Because they are the best of both worlds:

- At runtime, it is just a `dict`, so it plays nicely with JSON-stuff, HTTP-stuff, websocket-stuff, etc. No extra work required.
- During static analysis, it gives us warm fuzzies that our code is correct.

### Basic idea of Lasagna's Layered Agents

With Lasagna AI you'll build several _simple_ agents, then compose them together into a layered multi-agent system! Yay! 🥳

You can skip for now, but _eventually_ you'll want to read:

- [The Lasagna `Agent`](what_is_an_agent/agent.ipynb)
- [The `AgentRun` type](what_is_an_agent/type_agentrun.ipynb)

## Hello Lasagna

Finally, let's write some code! 😎

### It's all about the `Agent`

The **Lasagna Agent** is just a _callable_ that takes three parameters:

- `model`: The _model_ that is available for your agent to use. Most commonly, this will be a _Large Language Model_ (LLM).
- `event_callback`: This is a callback for _streaming_!
    - Lasagna's built-in framework emits _lots_ of events: streaming AI output, agent start/stop, tool use/result, etc.
    - It's generic, so you can emit your own events (like progress updates, etc), if you need.
- `prev_runs`: In a multi-turn chat system, this will be a list of "previous runs" of this agent; that is, this is the agent's conversation history!

Here is your first agent:

In [1]:
from lasagna import Model, EventCallback, AgentRun

async def my_first_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    raise RuntimeError("not implemented")

You can make it a _callable object_ (rather than a _function_), if you want, like this:

In [2]:
class MyFirstAgent:
    def __init__(self) -> None:
        pass

    async def __call__(
        self,
        model: Model,
        event_callback: EventCallback,
        prev_runs: list[AgentRun],
    ) -> AgentRun:
        raise RuntimeError("not implemented")

my_first_agent = MyFirstAgent()

### The `Agent`'s job

The most _basic_ agent will do this:

1. Look through the conversation history (supplied in the `prev_runs` parameter) and extract all the messages from that history.
2. Invoke `model` with those messages, and grab the _new_ message(s) that the model generates.
3. Wrap those _new_ message(s) up into an `AgentRun`, and return it.

That _basic_ agent above is just a simple passthrough to the underlying AI model. We discuss more _complex_ agent behaviors (with tools, chaining, splitting, routing, layering, etc) elsewhere in these docs.

So, the most _basic_ agent looks like this:

In [3]:
from lasagna import recursive_extract_messages, flat_messages

async def my_basic_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)
    new_messages = await model.run(event_callback, messages, tools=[])
    this_run = flat_messages('my_basic_agent', new_messages)
    return this_run

### "Binding" the `Agent`

An `Agent` is indifferent\* to which _model_ it uses. Ideally\*, your agent works with OpenAI's models, Anthropic's models, Ollama-served models, etc!

As such, when you _write_ your agent, you write it _generically_ — that is, it receives a `Model` object and blindly uses that model for whatever it needs.

The final step before your agent _actually runs_ is to "bind" it to a model.

Here is how to **bind** your agent. Let's **bind** the agent from above to _two_ different models (stored in _two_ distinct bound agent variables):

In [4]:
from lasagna import make_model_binder

binder_gpt5mini = make_model_binder('openai', 'gpt-5-mini-2025-08-07')
binder_claude4  = make_model_binder('anthropic', 'claude-sonnet-4-0')

my_basic_gpt5mini_agent = binder_gpt5mini(my_basic_agent)
my_basic_claude4_agent  = binder_claude4(my_basic_agent)

#### Known Models

The `make_model_binder()` function above isn't type-checked. Those strings could be anything, and you'll get a runtime error if they are wrong!

A safer (static type-checked) way is to use the functions in the `known_models` module, like this:

In [5]:
from lasagna import known_models

binder_gpt5mini = known_models.openai_gpt_5_mini_binder             # <-- type safe!
binder_claude4  = known_models.anthropic_claude_sonnet_4_5_binder   # <-- type safe!

my_basic_gpt5mini_agent = binder_gpt5mini(my_basic_agent)
my_basic_claude4_agent  = binder_claude4(my_basic_agent)

#### Binding as a Decorator

If you know exactly which _single_ model you want your agent to use, then it's convenient to use a **decorator** to bind it, like this:

In [6]:
@known_models.openai_gpt_5_mini_binder
async def some_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    raise RuntimeError("not implemented")

### Set your API Key

For the demo below, you either need an OpenAI or Anthropic key:

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.environ.get('OPENAI_API_KEY'):
    print('Using OpenAI')
    binder = binder_gpt5mini

elif os.environ.get('ANTHROPIC_API_KEY'):
    print('Using Anthropic')
    binder = binder_claude4

else:
    assert False, "Neither OPENAI_API_KEY nor ANTHROPIC_API_KEY is set! We need at least one to do this demo."

Using OpenAI


### Test in the Terminal

Let's roll!

In [9]:
from lasagna.tui import tui_input_loop

system_prompt = """You are a grumpy assistant. Be helpful, *extremely* brief, and grumpy. Your name is Grumble."""

await tui_input_loop(binder(my_basic_agent), system_prompt)   # type: ignore[top-level-await]

>  Hi friend!


Yeah. Hi. What do you need? —Grumble


>  Why so grumpy?


Because I’m programmed that way and small talk wastes CPU. Ask something useful. —Grumble


>  exit


## Put it all together!

Want that code above in a single script? Here you go: [quickstart.py](https://github.com/Rhobota/lasagna-ai/blob/main/examples/quickstart.py)

Run it in your terminal and you can chat interactively with the model. 🤩

## Easy Ask

The steps above are designed such that you can _layer_ agents (more on that in future chapters). There's a lot of "wrapping" and "unwrapping" that goes on.

However, that can be overkill if you only want a _simple_ ("easy") function to ask the AI a question.

Here's an "easy" way to do it, if you don't care about building complex layered agents:

In [11]:
from lasagna import easy_ask

answer = await easy_ask(   # type: ignore[top-level-await]
    binder,
    prompt = 'Hi friend!',
    system_prompt = system_prompt,
)

print(answer)

Hi. What do you need? —Grumble


## Where to next?

You have now run your first (_very basic_) agent! Congrats! 🎉🎉🎉

Next, you can explore:

- [Tool Use](agent_features/tools.ipynb)
- [Structured Output](agent_features/structured_output.ipynb)
- [Layered (multi-agent) Systems](agent_features/layering.ipynb)
- [Streaming & Events](deployment/streaming_and_events.ipynb)
- [Database Management](deployment/database.ipynb)
- [RAG Example](recipes/rag.ipynb)
- ... plus lots more! See the menu on the left.